In [ ]:
import numpy as np
import torch as pt
import torch.nn.functional as F

In [ ]:
# Load the data
TestStates = np.load("TestStates.npy")
dxNN = np.load("nnJumpLatVecs.npy")
RtoSiteInd = np.load("RtoSiteInd.npy")
SiteIndtoR = np.load("SiteIndtoR.npy")
GpermNNIdx = np.load("GroupNNpermutations.npy")

with open("GroupOpsIndices.pkl", "rb") as fl:
    GIndtoGDict = pickle.load(fl)

with open("supercellBCC.pkl", "rb") as fl:
    superBCC = pickle.load(fl)

permTensor = pt.tensor(GpermNNIdx).long()

# First layer test with Nch1 output channels

## First, construct the kernel

In [ ]:
# 4 channels
Nch1 = 4
psi = pt.rand(Nch1, 9)

In [ ]:
psi_repeat = pt.repeat_interleave(psi, permTensor.shape[0], dim=0)

In [ ]:
for i in range(0, 192, 48):
    for j in range(1, 48):
        assert pt.equal(psi_repeat[i], psi_repeat[i+j])

In [ ]:
permTensor_repeat = permTensor.repeat(Nch1, 1)
perm1 = permTensor_repeat[:48]
for i in range(48, 192, 48):
    perm2 = permTensor_repeat[i:i+48]
    assert pt.equal(perm1, perm2)

In [ ]:
# Now check rearrangements of nearest neighbors
psi_repeat_perm = pt.gather(psi_repeat, 1, permTensor_repeat)

## Next, let's test channelwise convolution

In [ ]:
state0 = pt.tensor(TestStates[0]/2.0).float()
outs0 = F.softplus(pt.matmul(psi_repeat_perm, state0).view(4, 48, 512))

In [ ]:
outs0_ch0 = outs0[0]

In [ ]:
out_ch0 = F.softplus(pt.matmul(psi_repeat_perm[:48], state0))

In [ ]:
assert pt.allclose(outs0_ch0, out_ch0)

In [ ]:
# Next do group averaging
outs0.shape

In [ ]:
outs0_avg = pt.sum(outs0, dim=1)/outs0.shape[1]

In [ ]:
# Check that this follows the symmetry of the original lattice